In [7]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import getopt
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import sys



if __name__ == "__main__":
  
  unixOptions = "sdt:edt"
  gnuOptions = ["start_dt=", "end_dt="]
  fullCmdArguments = sys.argv
  argumentList = fullCmdArguments[1:]

  try:
    arguments, values = getopt.getopt(argumentList, unixOptions, gnuOptions)
  except getopt.error as err:
    print (str(err))
    sys.exit(2)

  start_dt = ''
  end_dt = ''

  for currentArgument, currentValue in arguments:
    if currentArgument in ("-sdt", "--start_dt"):
      start_dt = currentValue
    elif currentArgument in ("-edt", "--end_dt"):
      end_dt = currentValue


  db_config = {'user':'my_user','pwd':'my_user_password','host':'localhost','port': 5432,'db':'zen'}

  connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],db_config['pwd'],db_config['host'],db_config['port'],db_config['db'])

  engine = create_engine(connection_string)
	        
  query = ''' SELECT 
                event_id,
                age_segment,
                event,
                item_id,
                item_topic,
                item_type,
                source_id,
                source_topic,
                source_type,
                TO_TIMESTAMP(ts / 1000) AT TIME ZONE 'Etc/UTC' as dt,
                user_id
            FROM log_raw
            WHERE (TO_TIMESTAMP(ts / 1000) AT TIME ZONE 'Etc/UTC') BETWEEN '{}'::TIMESTAMP AND '{}'::TIMESTAMP
        '''.format(start_dt, end_dt)


  log_raw = pd.io.sql.read_sql(query, con = engine, index_col = 'event_id')

  
  log_raw['dt'] = pd.to_datetime(log_raw['dt']).dt.round('min')

  #Параметры таблицы истории событий: 
  #Тема карточки (item_topic); Тема источника (source_topic); Возрастная категория (age_segment); Дата и время; Количество событий.

  dash_visits = log_raw.groupby(['item_topic','source_topic','age_segment','dt']).agg({'event':'count'}).rename(columns={'event':'visits'}).reset_index()

  
  #Параметры таблицы воронки: 
  #Тема карточки (item_topic); Возрастная категория (age_segment); Дата и время; Тип события (event); Количество уникальных пользователей.

  dash_engagement = log_raw.groupby(['dt','item_topic','event','age_segment']).agg({'user_id':'nunique'}).rename(columns={'user_id':'unique_users'}).reset_index()

  


  tables = {'dash_visits':dash_visits, 'dash_engagement':dash_engagement}
  for table_name, table_data in tables.items():
    query = '''
            DELETE FROM {} WHERE dt BETWEEN '{}'::TIMESTAMP AND '{}'::TIMESTAMP
          '''.format(table_name, start_dt, end_dt)
    engine.execute(query)
    table_data.to_sql(name = table_name, con = engine, if_exists = 'append', index = False)



  print(dash_visits.head())
  print(dash_engagement.head())

#запрос для терминала:
#python3 /Users/peter/Desktop/praktikum_dash_and_pipelines/zen_pipeline.py --start_dt='2019-09-24 18:00:00' --end_dt='2019-09-24 19:00:00'




In [9]:
dash_engagement

,record_id,dt,item_topic,event,age_segment,unique_users
0,11656,2019-09-24 18:28:00,Деньги,click,18-25,1
1,11657,2019-09-24 18:28:00,Деньги,click,31-35,1
2,11658,2019-09-24 18:28:00,Деньги,click,36-40,1
3,11659,2019-09-24 18:28:00,Деньги,show,18-25,13
4,11660,2019-09-24 18:28:00,Деньги,show,26-30,1
...,...,...,...,...,...,...
5653,17309,2019-09-24 19:00:00,Юмор,view,26-30,6
5654,17310,2019-09-24 19:00:00,Юмор,view,31-35,12
5655,17311,2019-09-24 19:00:00,Юмор,view,36-40,5
5656,17312,2019-09-24 19:00:00,Юмор,view,41-45,1


In [10]:
dash_visits

,record_id,item_topic,source_topic,age_segment,dt,visits
0,63102,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,63103,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,63104,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,63105,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,63106,Деньги,Авто,18-25,2019-09-24 18:56:00,27
...,...,...,...,...,...,...
30740,93842,Юмор,Финансы,36-40,2019-09-24 18:57:00,2
30741,93843,Юмор,Финансы,36-40,2019-09-24 19:00:00,1
30742,93844,Юмор,Финансы,41-45,2019-09-24 18:54:00,1
30743,93845,Юмор,Финансы,41-45,2019-09-24 18:56:00,1
